In [0]:
!nvidia-smi

In [0]:
import numpy as np
from PIL import Image
import tensorflow as tf
import cv2, math
from imageio import imsave
from scipy import signal
from matplotlib import pyplot as plt
from numba import njit, cuda, vectorize, guvectorize, stencil
from numba import prange
import cupy as cp
import math

Install additional profiling tools: line_profiler

In [0]:
!pip install line_profiler

In [0]:
%load_ext line_profiler

In a normal convolution , we **slide** the input filter spatially over the input image and compute **dot products**. Thus, it can be considered as a series of **elementwise multiplications** of filter with overalapping regions of inputs followed by an **addition** operation. Here, we limit ourselves to the most basic version of **2D** convolution on a grayscale image with a predefined **filter** with one **channel**(depth =1).

In [0]:
@njit
def convolution(mat, fil, res, stride=1):
    '''
    Performs convolution on a 2d square matrix.
    Expects a kernel with same height and width.

    '''
    row_range = col_range = len(mat)-len(fil)+1
    ker_sz=len(fil)
    for row in range(row_range):
        for col in range(col_range):

            # Elementwise multiplication of input submat with kernel 
            submat = mat[row:row+ker_sz, col:col+ker_sz]
            # Sum up the values and add bias
            respix = np.sum(submat*fil)

            res[row, col] = respix
    return res

@njit(parallel=True)
def convolution_parallel(mat, fil, res, stride=1):
    '''
    Performs convolution on a 2d square matrix.
    Expects a kernel with same height and width.

    '''
    row_range = col_range = len(mat)-len(fil)+1
    ker_sz=len(fil)
    for row in prange(row_range):
        for col in prange(col_range):

            # Elementwise multiplication of input submat with kernel 
            submat = mat[row:row+ker_sz, col:col+ker_sz]
            # Sum up the values and add bias
            respix = np.sum(submat*fil)

            res[row, col] = respix
    return res

Read a grayscale image of size 1024*1024 to work with 2D convolution

In [95]:
# Input image
img=cv2.imread('/content/lena.png',cv2.IMREAD_GRAYSCALE)
print("Image shape: {}".format(img.shape))

Image shape: (1024, 1024)


For simplicity, use stride=1 and set kernel width=kernel height. Now padding = (F - 1)//2 , where F is the filter size. Let's fix kernel size as 3x3 and then the padding value=1.

The general formula to find output size for convolution is:
OS = [(W−K+2P)/S]+1, where W is input, K is kernel, P is padding and S is stride. Now, in our case OS = [(1024-3 + 2*1)/1]=1023. In general if kernel and input is square and stride=1, output shape = W -1 if kernel-size is odd and W if it's even.

Let's set kernel size = (3x3)

In [96]:
#Input filter
fil= np.random.rand(3, 3).astype(np.float32)
print("Filter shape: {0}, Filter type: {1}".format(fil.shape,fil.dtype))

# Pad input if needed (value: 0, width: pad)
pad = (len(fil)-1)//2
img_pad = np.pad(img, pad, 'constant').astype(np.float32)
print("Image shape: {0}, Image type: {1} ".format(img_pad.shape, img_pad.dtype))

Filter shape: (3, 3), Filter type: float32
Image shape: (1026, 1026), Image type: float32 


Since we add padding, the output after convoltuion should have same size as original input image (i.e same padding), since it prevents overall shrinking.

In [97]:
# Output result (placeholder)
res = np.zeros(img.shape, dtype=np.float32)
print("Output shape: {0}, Output type: {1}".format(res.shape, res.dtype))

Output shape: (1024, 1024), Output type: float32


Let's run the convolution function without numba.The decorator 'njit' is used to compile the function just-in-time, in no=python mode. The original python function can be accessed as 'convolution.py_func'.

In [0]:
# convolve function without numba
results = convolution.py_func(img_pad, fil,res=res)
%timeit convolution.py_func(img_pad, fil,res=res)

Run the line profiler to find bottlenecks in the convolution function.

In [0]:
# Profile with line-profiler (use milli sec)
%lprun -u 1e-3 -f convolution.py_func convolution.py_func(img_pad,fil,res=res)

According to the results from line profiler, about **82%**(i.e 9.5/11.5 ms) of execution time is spent on the line: **respix = np.sum(submat*fil)**.

Now let's **speed-up** the convoultion using numb-jit.

In [99]:
# convolve function with numba
results = convolution(img_pad, fil,res=res) # Once the function is compiled and executed, the output is saved.
%timeit convolution(img_pad, fil,res=res) # Mesure average execution time

10 loops, best of 3: 118 ms per loop




We can add an additional argument 'parallel=True' to the jit decorator to exploit potential parallelization. In this case we also replace range with numba 'prange'  within the for loops of the function.

In [103]:
# convolve function with numba using parallel 
results = convolution_parallel(img_pad, fil,res=res)
%timeit convolution_parallel(img_pad, fil,res=res)

10 loops, best of 3: 26.5 ms per loop


In [0]:
# Run parallel diagnostics for profiling 
convolution_parallel.parallel_diagnostics(level=4)

Let's compute convolution using the scipy signal module. Here, we have to modify the kernel to adapt to the linear convolution behaviour in scipy to get the same results (Actually the ML convolution is correlation; not a real convolution). For this purpose, we flip the kernel along both the axes.

In [0]:
# scipy convolve
results_scipy = signal.convolve2d(img_pad, np.flip(fil, (0,1)), 'valid')
%timeit signal.convolve2d(img_pad, np.flip(fil, (0,1)), 'valid')

Lets' compare the outputs of the convolution using python-numba and scipy. Since all the python-numba used the same code(with differetn decorators), it would be sufficent to compare one numba version version output with scipy version.

In [0]:
# Verify outputs
print("Verification correct: ", np.allclose(results, results_scipy))

# Plot outputs
plt.imshow(results)
plt.show()

**Results:-**

Here is the **summary** of convolution operation performance in terms of **execution time**.

```
1.   Normal Convolution              - 6260 ms     
2.   Numba-JIT Convolution           -  125 ms
3.   Scipy Signal Convolution        -   42 ms
4.   Numba-JIT Parallel Convolution   -  28 ms
```

**Note:-**                                                                        
*   **CPU**: Intel(R) Xeon(R) CPU @ 2.30GHz, Dual-Core 
*   **Inputs**: 2D float32 matrices (i.e image: 1024x1024, filter: 3x3) 

**Optimization of convolution algorithm**

An **efficient** implementation of convolution usually involves a transformation of input matrix and filters into a unrolled or expanded layout, such that the **convolution** operation can then be performed as a **matrx multiplication** on the modified inputs.This transformation is usually referred to as '**im2col**'. Even though this makes the overall **convoltuion faster**; it significantly **increases the memory** requirement of the convolution process. Finally, matrix-multiplication outputs are converted back to the original input matrix representation using the **col2im** operation (i.e inverse of im2col).

**Note:** For simplicity, we set parameters like batch size, stride and number of channels as 1 and we perform padding as a pre-processing step to convolution operation.

In [0]:
@njit
def im2col_2d(mat, fil, res=1):
    '''
    Expects input and kernel to be square shape
    Returns : im2col view with shape - (ker_sz,ker_sz,img_sz,img_sz)
    '''
    # Parameters
    row_range = col_range = len(mat)-len(fil)+1
    ker_sz=len(fil)
    s0, s1 = mat.strides    
    shp = ker_sz,ker_sz,row_range,col_range
    strd = s0,s1,s0,s1

    out_view = np.lib.stride_tricks.as_strided(mat, shape=shp, strides=strd)
    return out_view

@guvectorize(['void(float32[:], float32[:,:], float32[:])'],'(n),(n,p)->(p)',target='parallel')
def matmulcore_cpu(A, B, C):
    n = len(A)
    n, p = B.shape
    for k in prange(n):
        for j in prange(p):
            C[j] += A[k] * B[k, j] 

First let's try out the classic numpy matmul(or dot in 2D) method along with im2col and col2im.

In [0]:
# Execute and save the result
col_mat=im2col_2d(img_pad, fil,res=1) # im2col reshape
mul_mat=np.matmul(fil.reshape(1,len(col_mat)*len(col_mat)), col_mat.reshape(len(col_mat)*len(col_mat),-1)) # matrix multiplication
results_numpy=mul_mat.reshape(len(img),len(img)) # col2im reshape

In [0]:
# Mesure execution time with timeit
def numpy_conv(img_pad, fil,res=1):
    col_mat=im2col_2d(img_pad, fil,res=1) # im2col reshape
    mul_mat=np.matmul(fil.reshape(1,len(col_mat)*len(col_mat)), col_mat.reshape(len(col_mat)*len(col_mat),-1)) # matrix multiplication
    results_numpy=mul_mat.reshape(len(img),len(img)) # col2im reshape

%timeit numpy_conv(img_pad, fil,res=1)

In [0]:
# Profile with line-profiler (use milli sec)
%lprun -u 1e-3 -f numpy_conv numpy_conv(img_pad, fil,res=1)

From the line profiler log, it is clear that **99.7%** of the time is used in **np.matul**, i.e **im2col and col2im** operations have **negligible** execution time (**Caching** could also be the reason).

Now, lets execute the numba version of convolution. Here, we use a vectorized parallel version of matrix multiplication instead of built-in numpy function.

In [0]:
# Compile, execute and save the output
C=np.zeros(len(img)*len(img), dtype=np.float32)
col_mat=im2col_2d(img_pad, fil,res=1) # im2col reshape
colres=matmulcore_cpu(fil.reshape(1,len(col_mat)*len(col_mat)), col_mat.reshape(len(col_mat)*len(col_mat),-1), C) # matrix multiplication
results_matmulcpu=colres.reshape(len(img),len(img)) # col2im reshape

In [0]:
%%timeit
C=np.zeros(len(img)*len(img), dtype=np.float32)
col_mat=im2col_2d(img_pad, fil,res=1) # im2col reshape
colres=matmulcore_cpu(fil.reshape(1,len(col_mat)*len(col_mat)), col_mat.reshape(len(col_mat)*len(col_mat),-1), C) # matrix multiplication
results_matmulcpu=colres.reshape(len(img),len(img)) # col2im reshape

This, time it seems buit-in **numpy function** is slightly **faster** than our numba-jitted function.

Let's compare the results of numpy and numba version of the new optimized convolution algoritm.

In [0]:
# Verify outputs
print("Verification CPU correct : ", np.allclose(results_numpy, results_matmulcpu))

# Plot outputs
plt.imshow(results_matmulcpu)
plt.show()

**Results:-**

Here is the performance **summary** of the optimized convolution algorithm with **im2col and col2im**.

```
1.   Numpy Im2Col Convolution               -    8 ms     
2.   Numba-JIT Parallel Im2Col Convolution  -   13 ms
```

**Note:-**                                                                        
*   **CPU**: Intel(R) Xeon(R) CPU @ 2.30GHz, Dual-Core 
*   **Inputs**: 2D float32 matrices (i.e image: 1024x1024, filter: 3x3) 

**Convolution in GPU: Cupy**

CuPy is an open-source matrix library accelerated with CUDA. It uses libraries like CUBLAS,CuFFT and CuDNN for acclerating matrix operations under the hood. It's interface is highly compatible with numpy and in most cases you can directly replace the numpy import with cupy.

In [0]:
@njit
def im2col_2d(mat, fil, res=1):
    '''
    Expects input and kernel to be square shape
    Returns : im2col view with shape - (ker_sz,ker_sz,img_sz,img_sz)
    '''
    # Parameters
    row_range = col_range = len(mat)-len(fil)+1
    ker_sz=len(fil)
    s0, s1 = mat.strides    
    shp = ker_sz,ker_sz,row_range,col_range
    strd = s0,s1,s0,s1

    out_view = np.lib.stride_tricks.as_strided(mat, shape=shp, strides=strd)
    return out_view

Initially transform input using **im2col**. Then, **copy** the inputs from host to **device memory** i.e GPU. 




In [0]:
# Convert inputs using im2col in CPU
col_mat=im2col_2d(img_pad, fil,res=1) # im2col reshape

# Load inputs and allocate output memory in GPU
fil_GPU=cp.asarray(fil.reshape(1,len(col_mat)*len(col_mat))) # (1,9)
mat_GPU=cp.asarray(col_mat.reshape(len(col_mat)*len(col_mat),-1)) # (9, 1048576)
out_GPU=cp.asarray(np.zeros((len(img),len(img)), dtype=np.float32)) # (1024,1024)

print("Output device : {0}, Data type: {1}, Shape: {2}".format(out_GPU.device, out_GPU.dtype, out_GPU.shape))

Now you can perform **dot product** directly using cupy. Since cupy syntax is very similar to numpy, **col2im_2D** operation can be performed using **reshape** operator.

In [0]:
# Execute dot product on GPU and reshape output(col2im_2D)
out_GPU=cp.dot(fil_GPU,mat_GPU).reshape(1024,1024)

In [0]:
# Measure execution time on GPU, using cupy
%%timeit
out_GPU=cp.dot(fil_GPU,mat_GPU).reshape(1024,1024)
cp.cuda.Device().synchronize() # Do explicit synchronize to prevent asynchronous execution

Thus cupy(GPU) gives around 30x speed improvent compared to our previous best result of 8 ms, using numpy built-in functions. Remember, that the built-in numpy and cupy functions are highly optimized using BLAS libraries.

In [0]:
# Copy results back to cpu memory
results_cupy=cp.asnumpy(out_GPU)
results_cupy.shape

Finally, copy the results back to host memory and compare the results.

In [0]:
# Verify outputs
print("Verification Cupy correct : ", np.allclose(results, results_cupy))

# Plot outputs
plt.imshow(results_cupy)
plt.show()

**Convolution in GPU: Numba CUDA**


The **CUDA JIT** is a low-level entry point to the CUDA features in Numba. It translates Python functions into **PTX code** which execute on the CUDA hardware. We can use a **cuda.jit** decorator, just like CPU, to compile our python code and make it run on GPU. However, you have to configure a hierarchy of '**grid of thread blocks**' and modify your code according to the parallel programming model prescribed by **CUDA API**, with the help of numba functions.


In [0]:
@njit
def im2col_2d(mat, fil, res=1):
    '''
    Expects input and kernel to be square shape
    Returns : im2col view with shape - (ker_sz,ker_sz,img_sz,img_sz)
    '''
    # Parameters
    row_range = col_range = len(mat)-len(fil)+1
    ker_sz=len(fil)
    s0, s1 = mat.strides    
    shp = ker_sz,ker_sz,row_range,col_range
    strd = s0,s1,s0,s1

    out_view = np.lib.stride_tricks.as_strided(mat, shape=shp, strides=strd)
    return out_view


@cuda.jit('(float32[:,:], float32[:,:], float32[:,:])')
def matmulcore_gpu(A, B, C):

    i, j = cuda.grid(2)
    if i < C.shape[0] and j < C.shape[1]:
        tmp = 0.
        for k in range(A.shape[1]):
            tmp += A[i, k] * B[k, j]
        C[i, j] = tmp

Convert the inputs using **im2col** and load it into **GPU memory**. Also, allocate global memory on **GPU** to save the **outputs** results. Once the kernel is compiled and executed, the results will be stored in this GPU memory. Now, configure the **grid and block** size according to your input shape and size.

In [0]:
# Convert inputs using im2col in CPU
col_mat=im2col_2d(img_pad, fil,res=1) # im2col reshape

# Load inputs and allocate memory for output in GPU (globally)
A_GPU=cuda.to_device(fil.reshape(1,len(col_mat)*len(col_mat))) # (1,9)
B_GPU=cuda.to_device(col_mat.reshape(len(col_mat)*len(col_mat),-1)) # (9, 1048576)
C_GPU=cuda.to_device(np.zeros((1,len(img)*len(img)), dtype=np.float32)) # (1,1048576)

# Configure thread blocks for CUDA
threadsperblock = (1, 256)
blockspergrid_x = int(math.ceil(A_GPU.shape[0] / threadsperblock[0]))
blockspergrid_y = int(math.ceil(B_GPU.shape[1] / threadsperblock[1]))
blockspergrid = (blockspergrid_x, blockspergrid_y)

print("Blocks per gird: {}".format(blockspergrid))

Here we have **4096** blocks in total. Each blocks have **256** thread in them.
So, 4096*256 = 1048576 => Output size.

**Note:** Here the 2D dimension(1) is redundant, since our output and input filter are just 1D vectors in essence. The extra axis is for the sake of generalization demo.

In [0]:
# Execute and benchmark the GPU kernel execution time
%%timeit
matmulcore_gpu[blockspergrid, threadsperblock](A_GPU,B_GPU, C_GPU)
cuda.synchronize()

**Note:** By default, the kernel executes **asynchronously** with respect to host. So, add an explicit '**synchronize**' function to make sure that the **timeit** measures the actual **execution time** of the kernel.

In [0]:
# Copy results back to cpu memory
results_numba_gpu=C_GPU.copy_to_host()
results_numba_gpu=results_numba_gpu.reshape(1024,1024) # col2im2D
results_numba_gpu.shape

Copy the results back to **host memory** and **reshape** the ouputs(col2im_2D). Finally, **compare** the outputs and plot the results.

In [0]:
# Verify outputs
print("Verification Numba GPU correct : ", np.allclose(results, results_numba_gpu))

# Plot outputs
plt.imshow(results_numba_gpu)
plt.show()

**Experiment:** For further **speedup** we can use techniques like **cuda streams and shared memory** etc.

**Results:-**

Here is the performance **summary** of the optimized convolution algorithm with **im2col and col2im**.

```
1.   Cupy Im2Col Convolution            -   256 μs     
2.   Numba-JIT CUDA Im2Col Convolution  -   230 μs
```

**Note:-**                                                                        
*   **CPU**: Intel(R) Xeon(R) CPU @ 2.30GHz, Dual-Core 
*   **GPU**: Nvidia Tesla T4, CUDA Version: 10.1 
*   **Inputs**: 2D float32 matrices (i.e image: 1024x1024, filter: 3x3) 
*   Host-to-device/Device-to-host **memory copy time** is not included in timing. 

**XNOR Convolution** 

The XNOR convolution used in binary neural networks, has both the **inputs and filters in binarized format**. Using this approximate representation, we can compute convolution using **XNOR and bitcounting** operations instead of traditional MAC operations. Even though there would be a drop in accuracy, the advantages include **speed-up, memory savings and reducd power** requirements.

In [0]:
@njit
def im2col_2d(mat, fil, res=1):
    '''
    Expects input and kernel to be square shape
    Returns : im2col view with shape - (ker_sz,ker_sz,img_sz,img_sz)
    '''
    # Parameters
    row_range = col_range = len(mat)-len(fil)+1
    ker_sz=len(fil)
    s0, s1 = mat.strides    
    shp = ker_sz,ker_sz,row_range,col_range
    strd = s0,s1,s0,s1

    out_view = np.lib.stride_tricks.as_strided(mat, shape=shp, strides=strd)
    
    return out_view

In [0]:
# Initialize inputs and filters randomly
img_pad=np.random.uniform(low=-1.0, high=1.0, size=(1026,1026)).astype(np.float32)
fil=np.random.uniform(low=-1.0, high=1.0, size=(3,3)).astype(np.float32)


Initialy, we convert the inputs and filters with **im2col** operation and then 
**binarize** them to 0 and 1(or 1 & -1).

In [0]:
col_mat=im2col_2d(img_pad, fil,res=1) 
col_mat = col_mat.reshape(len(col_mat)*len(col_mat),-1) #(9, 1048576)

In [0]:
# im2col reshape
col_mat=im2col_2d(img_pad, fil,res=1) 
col_mat = col_mat.reshape(len(col_mat)*len(col_mat),-1) #(9, 1048576)
fil_mat=fil.reshape(1,len(fil)*len(fil)) # (1, 9)

# Binarize col_mat and fil_mat
bin_col=np.uint8(col_mat>0)
bin_fil=np.uint8(fil_mat>0)

Here is the comple **formula for XNOR** convolution:-


*   **W**  -  real valued weights (filter)
*   **I**  -  real valued input tensor
*   **\***  -  normal convolution operation

    **`I ∗ W ≈ H ⊛ B ⊙ αK`**

* **H** – input binary tensor (the sign of I)
* **B** – binary weights (the sign of W)
* **⊛** is a convolution using only XNOR and bit-counting
* **α** – real valued scale factors (the average of |W|)
* **K** – real valued scale factors of input conv-windows(absolute values)
* **⊙** is elementwise multiplication

In [0]:
# Calculate scale factors K and alpha
K=np.mean(np.abs(col_mat),axis=0) # float32 (1048576,)
alpha=np.mean(np.abs(fil_mat)) # foat32 ()

Compare execution time for calculation of **K**, between python and numba versions.

In [0]:
# Parallel mean on array after im2col with reduction
z=np.zeros(1048576,dtype=np.float32)
@njit(parallel=True)
def fmean(arr,z):
    for i in prange(1048576):
      for j in prange(9):
       z[i]+=np.abs(arr[j,i])
    return z/9


# Parallel mean on array after im2col with vectorization
@guvectorize(['(float32[:], float32[:])'],'(n)->()', target='parallel')
def guvec_avg(array, out):
    acc = 0
    n = len(array.T)
    for val in array:
        acc += np.abs(val)
    out[0] = acc/n


# Parallel mean on array before im2col with stencil
@stencil
def kernell(a):
    return (abs(a[0, 0]) + abs(a[0, 1]) + abs(a[0, 2]) 
          + abs(a[1, 0]) + abs(a[1, 1]) + abs(a[1, 2])
          + abs(a[2, 0]) + abs(a[2, 1]) + abs(a[2, 2]))/9

@njit(parallel=True)
def stencil_avg(img_pad):
    return kernell(img_pad)

# Get results
avg=stencil_avg(np.abs(img_pad))
stencil_version=avg[:1024,:1024].reshape(-1)
im2col_version=fmean(col_mat,z)
guvec_version = guvec_avg(col_mat.T)
pyvecd_version=np.mean(np.abs(col_mat),axis=0,dtype=np.float32)

# Compare execution time
%timeit stencil_avg(img_pad)
%timeit fmean(col_mat,z)
%timeit np.mean(np.abs(col_mat),axis=0,dtype=np.float32)
%timeit guvec_avg(col_mat.T)

**Note:** Both @vectorize and @guvectorize parallelize on the **broadcast dimensions** in a ufunc/gufunc.

Perform **xnor** betwen binary filter and binary input array.

In [9]:
# calculate xnor between col and fil
out_xnor=np.logical_not(np.logical_xor(bin_fil.reshape((-1,1)),bin_col))
print("Out shape: {}, Out type: {}".format(out_xnor.shape, out_xnor.dtype))

Out shape: (9, 1048576), Out type: bool


After performing XNOR operation, we need to apply **bitcount** over individual columns of binary numbers. The final result is obtained using the formula,

**`res = 2*popcount - num(bits)`**

Here **popcoun**t refers to the number of setbits of binary input. Since we have 0's and 1's we can just **sum** up the values across the columns for computing popcount.

In [0]:
# Calculate bitcount using formula b = 2*popcount - num(bits)
out = 2*(np.sum(out_xnor,axis=0)) - len(out_xnor)

# Multiply result with scaling factor
res = out*K*alpha

Now, **multiply** the result with **K and alpha elementwis**e for **scaling** the output values and  finally **reshape** the output according to the original (calculated) output shape.

In [0]:
# Reshape the output 
xnor_result= res.reshape((1024,1024))

**NB**: Here, we limit ourselves to **2D inputs** for sake of simplicity. In general we have to take **averages** across all the **channels** for calculating **K** values.

**XNOR Convolution With Bitpacking**

In the previous example, even though we converted our inputs to **binray format** (0/1), they were still occupying **64 bits** (32 bits if inputs are float32) in memory. We could also have converted the binary inputs of 0's and 1's into **UINT8** format for saving memory.

Here, we use another technique called **bitpacking** for compressing our inputs in memory. We **group** together consecutive **64** elements (0 or 1) into a single **UINT64** number. In our transformed input matrix (bin_col: (1048576, 9)), we group **consecutive rows** together such that new row length approximately becomes 64.

`Reshape => 7 consecutive rows of 9 elements = 63 bits or elements = a new row.`

Initially, we have to **pad input** mat with zero row vectors (i.e **3** nos,since 1048576mod7=4) to complete the **last batch** or group of 7 rows. For each row **pad** the remaining positions to make its length 64(i.e add a zero at the end of each row). In the case of input filter, we **repeat** the kernel **7 times** and then pad it with a zero. Now, the kernel row vector and each rows of the input have 64 binary elements. 


In [0]:
#Get binarized inputs 
bin_fil=bin_fil.copy() # shape: (1, 9)
bin_col=bin_col.T.copy() # shape: (1048576, 9)

Our aim is to pack a group of **consecutive elements** (each element UINT8/Float/Bool of 0 or 1) into a **single 64 bit UINT64** element, such that our **kernel** becomes **single number**(0D) and **input** a **1D vector**, both of type UINT64.

In [0]:
def bitpack(bin_fil,bin_col):
  
  # Bitpack filter
  a=bin_fil.repeat(7,axis=0)
  a_64=np.append(a, [0]) # (64,) 
  fil_pack=np.packbits(a_64).view(np.uint64).byteswap()
  
  # Bitpack image
  row_pad=np.zeros((3,9),dtype=np.uint8) 
  b=np.concatenate([bin_col,row_pad],axis=0) # pad col end(1048576 to 1048579)
  b=b.reshape((-1,63))

  col_pad=np.zeros((b.shape[0],1),dtype=np.uint8)
  b=np.concatenate([b,col_pad],axis=1) # pad row end(63 to 64)
  col_pack=np.packbits(b).view(np.uint64).byteswap()
  
  return fil_pack, col_pack

# Execute and save the results
fil_pack,col_pack=bitpack(bin_fil,bin_col)

print("fil_pack: shape= {0}, type= {1}".format(fil_pack.shape, fil_pack.dtype))
print("col_pack: shape= {0}, type= {1}".format(len(col_pack),col_pack.dtype))

# Compute average execution time
%timeit bitpack(bin_fil,bin_col)

**Note**: Reshaping a large **non-contiguous** array(ie non-C_CONTIGUOUS) consumes significant amount of time compared to it's contiguous version, so verify them using **flags** properties of array. This situation sometimes arises if we use reshape on a **view or transposed** version of a numpy array.

In [0]:
# Comapre numpy bitpack wiht numba bitpack
test = np.random.randint(0, 2, (149797, 64),dtype=np.uint8)
@njit(parallel=True)
def shifting(bitlist): 
    rows,cols=149797,64
    out=0
    z=np.zeros(rows,dtype=np.uint64)
    for i in prange(rows):
      for bit in range(cols):
         z[i] = (z[i] * 2) + bitlist[i,bit] 

    return z

%timeit shifting(test)
%timeit np.packbits(test).view(np.uint64).byteswap()

**Note**: The **performance** depends upon your hardware(more cores,clocks speed => numba parallel better) and **library versions**(highly optimized numpy), input size etc. Using a **12 core** machine, i got **10x** speed-up for numba version, in comparison to numpy pack-bits. Anyway, in google **colab with dual core cpu** and default numpy, the pure **numpy version** seems to be faster.



Finally we perform **bitwise XNOR** of kernel across all elements of input.

In [0]:
# Perform XNOR between inputs
z=np.bitwise_not(np.bitwise_xor(fil_pack,col_pack))
%timeit np.bitwise_not(np.bitwise_xor(fil_pack,col_pack))

Now, **z** is an **UINT64** array of size **149797**

In [0]:
# Classic C-Style bit-count for unpacking
@njit
def bit_count(n):
    """Return the number of bits set to 1 in the integer number 'n'.
       This is called the Hamming weight or the population count of 'n'.
       The algorithm performs as many iterations as there are set bits.
       Argument 'n' must be non-negative'
    """
    count = 0
    while n:
        n &= n - np.uint64(1)
        count += 1
    return count

# Nine-bit extractor for unpacking
@njit
def bit_ext():
  '''Compute and save numbers for extracting nine bits
     from binary representation of a UINT64 number.
  '''

  # Initialize bit extractor
  bits=np.zeros(7,dtype=np.uint64)
  ext9=18410715276690587648# FF800000
  for i in range(0,7):
    pos=i*9
    bits[i]=ext9 >> pos
  return bits

# Get bit extractor
bits=bit_ext()

# Compute average execution time
%timeit bit_ext

After performing XNOR, we have to **unpack** the result from the individual elements of the array.  Each **64 bit element** has to be unpacked to **7 float numbers**(7*9+ 1 pad =64), using the unpack function. 

Here,for each consecutive **nine bits**, we calculate the output using the aforementioned **bitcount formula**.

In [0]:
@njit
def unpack(z):
 '''input: int64
    output: float array of length 7 (7*9 packs)
 '''
 fout=np.zeros((1048579),dtype=np.float32)
 num=np.zeros((7),dtype=np.float32)

 for i in range(0,len(z)):
   for j in range(0,7):
     popcount=bit_count(z[i] & bits[j])
     num[j]=2*popcount - 9
   pos=i*7  
   fout[pos:pos+7]=num
   
 return fout

# Unpack and save result
final=unpack(z)

# Compute average executin time
%timeit unpack(z)

Flatten the output and remove the **last 3 elements** corresponding to padded zero row vectors. Now, **multiply** the output with **K and alpha** elementwise and **reshape** the output according to the calculated output shape.

In [0]:
# Scale the output using K and alpha 
@njit
def scale(mat, K, alpha):
  scale=mat*K*alpha
  return scale

# Scale and reshape output
xnor_bitpack=scale(final[0:-3],K,alpha).reshape((1024,1024))

# Compute average execution time
%timeit scale(final[0:-3],K,alpha).reshape((1024,1024))

Verify the results by **comparing** it with previous outputs.

In [0]:
print("Verification bitpack correct : ", np.allclose(xnor_bitpack, xnor_result))

**Results:-**

Here is the performance summary of the optimized convolution algorithm with **xnor and bitpacking**.

```
Im2col   : 4.96 ms
Binarize : 4.86 ms
K-alpha  : 2.70 ms
Bitpack  : 4.53 ms
XNOR     : 0.32 ms
Bitunpack: 1.75 ms
Scale    : 0.56 ms
------------------
i.e Total time = 20 ms
```

**Note:-**                                                                        
*   **CPU**: Intel(R) Xeon(R) CPU @ 2.30GHz, Dual-Core 
*   **Inputs**: 2D float32 matrices (i.e image: 1024x1024, filter: 3x3)
*   **Parallelization** speed-up depends on hardware and data size.
*   **Operations** on non-contiguous array consumes significant amount of time.
*   **Im2col and binarization** seems to be the current bottlenecks.
*   Need **finer control** over memory (bit-level and hierarchy).
*   Need **generalization** and lesser overheads.
*   **Execution time** seems to vary between different colab runtime's.

**Numba: Stencil Convoltuion**

Stencils are a common computational **pattern** in which array elements are updated according to some fixed pattern called the stencil kernel. Each element produces an output that is the **dot product** of the kernel and the 3x3 subarray neighbourhood. Using the **parallel-jit** option, we can execute the whole process in parallel with **numba stencils**.

In [0]:
# Input image
img=cv2.imread('/content/lena.png',cv2.IMREAD_GRAYSCALE)
print("Image shape: {}".format(img.shape))

#Input filter
fil= np.random.rand(3, 3).astype(np.float32)
print("Filter shape: {0}, Filter type: {1}".format(fil.shape,fil.dtype))

# Pad input if needed (value: 0, width: pad)
pad = (len(fil)-1)//2
img_pad = np.pad(img, pad, 'constant').astype(np.float32)
print("Image shape: {0}, Image type: {1} ".format(img_pad.shape, img_pad.dtype))

# Output result (placeholder)
res = np.zeros(img.shape, dtype=np.float32)
print("Output shape: {0}, Output type: {1}".format(res.shape, res.dtype))


In [0]:
# Parallel conv on image with stencil
@stencil
def kernel_conv(a):
    return (a[0, 0]*fil[0,0] + a[0, 1]*fil[0,1] + a[0, 2]*fil[0,2] 
          + a[1, 0]*fil[1,0] + a[1, 1]*fil[1,1] + a[1, 2]*fil[1,2]
          + a[2, 0]*fil[2,0] + a[2, 1]*fil[2,1] + a[2, 2]*fil[2,2])

@njit(parallel=True)
def stencil_conv(img_pad):
    return kernel_conv(img_pad)

**Note:** Currently the kernel is accessed as a global variable inside stencil.

In [0]:
# Compare stencil convolution with scipy convolution
stencil_cpu=stencil_conv(img_pad)[:1024,:1024]
scipy_res = signal.convolve2d(img_pad, np.flip(fil.copy(), (0,1)), 'valid')

%timeit stencil_conv(img_pad)
np.allclose(stencil_cpu,scipy_res)

Here is the corresponding **GPU** version of the code, using generic stencils. Note that we need to configure the **grid of thread blocks** before running the kernels in GPU.

In [0]:
@cuda.jit
def convolve_gpu(x, k, out):
    i, j = cuda.grid(2)
    n, m = x.shape
    if i < out.shape[0] and j < out.shape[1]:
        out[i, j] = (x[i, j]  *k[0, 0] + x[i, j+1]  *k[0, 1]  + x[i, j+2]  *k[0, 2] +
                     x[i+1, j]*k[1, 0] + x[i+1, j+1]*k[1, 1]  + x[i+1, j+2]*k[1, 2] +
                     x[i+2, j]*k[2, 0] + x[i+2, j+1]*k[2, 1]  + x[i+2, j+2]*k[2, 2])


# Allocate memory for inputs and outputs
fil_gpu=cp.asarray(fil)
mat_gpu=cp.asarray(img_pad)
out_gpu=cp.asarray(np.zeros(img.shape, dtype=np.float32))

# Configure the grid of thread blocks
threadsperblock = (32,32)
blockspergrid_x = math.ceil(mat_gpu.shape[0] / threadsperblock[0])
blockspergrid_y = math.ceil(mat_gpu.shape[1] / threadsperblock[1])
blockspergrid = (blockspergrid_x, blockspergrid_y)

# Execute the convolution kernel
convolve_gpu[blockspergrid, threadsperblock](mat_gpu, fil_gpu ,out_gpu)

# Copy the results back to host and compare results
stencil_gpu = cp.asnumpy(out_gpu)
np.allclose(stencil_cpu,stencil_gpu)

In [0]:
## Measure execution time for gpu-stencil
%%timeit 
convolve_gpu[blockspergrid, threadsperblock](mat_gpu, fil_gpu ,out_gpu)
cuda.synchronize()

**Results:-**

Here is the performance **summary** of the convolution algorithm with **stencils**.

```
1.   Numba CPU Stencil  -   3620 μs     
2.   Numba GPU Stencil  -    740 μs
```

**Note:-**                                                                        
*   **CPU**: Intel(R) Xeon(R) CPU @ 2.00GHz
*   **GPU**: Nvidia Tesla P100, CUDA Version: 10.1 
*   **Inputs**: 2D float32 matrices (i.e image: 1024x1024, filter: 3x3) 
*   Host-to-device/Device-to-host **memory copy time** is not included in timing. 

Finally, let's see performance of **tensorflow** convolution ...

In [0]:
conv_layer = tf.keras.layers.Conv2D(1, 3)

@tf.function
def conv_fn(image):
  return conv_layer(image)

image = tf.zeros([1, 1026, 1026, 1]) # NHWC
# warm up
res=conv_fn(image)

print("Tensorflow conv:", timeit.timeit(lambda: conv_fn(image), number=1000), "ms per loop")

It seem to be around **1 ms** in GPU !!!

It surely is close to our **best results**(stand-alone) so far, considering all the potential tensorflow **overheads** ...